# Introduction to Python

# Homework #4
# Due Tuesday Oct 20 , 11:50pm 


# Problem 1 - Longest Ordered Substring(los)
- argument: a string
- returns: the longest substring such that 
the characters strictly increase(in the < sense) from 
left to right
- if there is more than one longest string, return any one of them
  

In [1]:
# usual ordering
['a' < 'b', 'b' < 'a', 'a' < 'a']

[True, False, False]

In [17]:
def los(string):
    longest = string[0]
    current = string[0]
    for i in string[1:]:
        if current[-1] < i:
            current += i
            if len(longest) < len(current):
                longest = current
        else:
            current = i
    return(longest)

In [18]:
los('abcd')

'abcd'

In [19]:
los('xabcyz')

'abcyz'

In [20]:
los('cba')

'c'

In [21]:
los('larry')

'ar'

In [22]:
los('xabcxabcdexuvwxyz')

'abcdex'

In [23]:
los('xabcxabcdexpuvwxyz')

'puvwxyz'

# Problem 2
- suppose we want to convert between C(Celsius) and F(Fahrenheit), using the equation 9*C = 5*(F-32)
- write functions 'c2f' and 'f2c'
- do all computation in floating point for this problem

In [30]:
from __future__ import division
def c2f(Celsius):
    Fahrenheit = Celsius*9/5 + 32
    return(Fahrenheit)
def f2c(Fahrenheit):
    Celsius = 5*(Fahrenheit-32) / 9
    return(Celsius)

In [31]:
[c2f(0), c2f(100), f2c(32), f2c(212)]

[32.0, 212.0, 0.0, 100.0]

- to write f2c, you solved the equation for C, and made a function out of the other side of the equation
- to write c2f, you solved for F, ...
- there is another way to think about this 
- rearrange the equation into a symmetric form

```
9*C - 5*F = -32*5
```

- you can think of the equation above as a "constraint" between F and C. if you specify one variable, 
the other's value is determined by the equation. in general, if we have

```
c0*x0 + c1*x1 + ... cN*xN = total
```

- cI are fixed coefficients
- specifying any N of the (N+1) x's will determine the remaining x variable
- define a class, 'Constaint' that will do 'constraint satisfaction'

In [100]:
import numpy as np
from sympy.solvers import solve
from sympy import Symbol

In [177]:
class Constraint:
    def __init__(self, symbol, array1, res):
        self.l = symbol.split(' ')
        self.ll = self.l.copy()
        self.array1 = np.array(array1, float)
        self.res = res
    def setvar(self, S, value):
        if isinstance(S, str):
            self.l = [value if i == S else i for i in self.l]
        else:
            self.l[S] = value
        array2 = list(map(lambda x: Symbol(x) if isinstance(x, str) else x, self.l))
        array2 = np.array(array2)
        out = solve(sum(self.array1*array2)-self.res, dict=True)
        flag = list(map(lambda x: type(x), self.l))
        if flag.count(str) == 1:
            self.l = self.ll
        return(out)

In [178]:
# setup constraint btw C and F
# 1st arg is var names, 
# 2nd arg is coefficients
# 3rd arg is total

c = Constraint('C F', [9,-5], -5*32)

# 1st arg - variable index or name
# 2nd arg - variable value
# setvar will fire when there is only one unset variable remaining
# it will print the variable values, and returns them in a list, and
# clear all variable values
c.setvar(0, 100)

[{F: 212.000000000000}]

In [179]:

# can set var by index or name
c.setvar('C', 0)

[{F: 32.0000000000000}]

In [180]:
c.setvar('F', 212)

[{C: 100.000000000000}]

In [181]:
c2 = Constraint('x0 x1 x2 x3 x4', range(5), 1)
c2.setvar('x0', 0)

[{x1: -2.0*x2 - 3.0*x3 - 4.0*x4 + 1.0}]

In [182]:
c2.setvar('x1', 10)

[{x2: -1.5*x3 - 2.0*x4 - 4.5}]

In [183]:
# x2
c2.setvar(2,20)

[{x3: -1.33333333333333*x4 - 16.3333333333333}]

In [184]:
# only one unset var left, x4, so fire
c2.setvar('x3', 30)

[{x4: -34.7500000000000}]

# Problem 3
- write 'mindot'. given two equal length vectors, different dot products can
be calculated, by permutting the order of the vectors. for example, given the
vectors
```
[10,20]
[0, 1]
```
- there are two possible dot products, 10 and 20. 
- mindot should return the min, 10
- mindot can be written very easily by using functions from itertools and functools
- 'operator' module has functional versions of operators
  - '+' <=> operator.add
  - '*' <=> operator.mul

In [1]:
import itertools 
import functools
import operator

v2a = [10,20]
v2b = [0, 1]

v3a = [1,3,-5]
v3b = [-2, 4, 1]

v4a = range(1,6)
v4b = [1,0,1,0,1]

[operator.add(2,3), operator.mul(2,3)]

[5, 6]

In [24]:
def dotproduct(vec1, vec2, sum=sum, map=map, mul=operator.mul):
    return sum(map(mul, vec1, vec2))
def mindot(vec1, vec2):
    List = []
    for i in itertools.permutations(vec2, len(vec2)):
        l = dotproduct(vec1, i)
        List.append(l)
    return(min(List))

In [25]:
[mindot(v2a,v2b),mindot(v3a, v3b), mindot(v4a, v4b)]

[10, -25, 6]

# Problem 4
- You are in a store, and you have some cash burning a hole in your pocket - you want to spend all of it!!
- write 'pickitems' 
  - 1st arg - list of prices for things in the store
  - 2nd arg - cash you have
  - returns - list of prices that will exactly spend your cash
- itertools module is your friend

In [32]:
cash1 = 4
prices1= [1,1,1,1,8]

cash2 = 200
prices2 = [150, 24, 79, 50, 88, 345, 3]

cash3 = 8
prices3 = [2, 1, 9, 4, 4, 56, 90, 3]

cash4 = 542
prices4 = [230, 863, 916, 585, 981, 404, 316, 785, 
       88, 12, 70, 435, 384, 778, 887, 755, 740, 
       337, 86, 92, 325, 422, 815, 650, 920, 125,
       277, 336, 221, 847, 168, 23, 677, 61, 400,
       136, 874, 363, 394, 199, 863, 997, 794, 587,
       124, 321, 212, 957, 764, 173, 314, 422, 927,
       783, 930, 282, 306, 506, 44, 926, 691, 568,
       68, 730, 933, 737, 531, 180, 414, 751, 28, 
       546, 60, 371, 493, 370, 527, 387, 43, 541,
       13, 457, 328, 227, 652, 365, 430, 803, 59,
       858, 538, 427, 583, 368, 375, 173, 809, 896,
       370, 789]


In [33]:
def pickitems(prices, cash):
    for i in range(1,len(prices)+1):
        for r in itertools.combinations(prices, i):
            if sum(r) == cash:
                return(r)

In [34]:
[pickitems(prices1, cash1), pickitems(prices2, cash2), pickitems(prices3, cash3), pickitems(prices4, cash4)]

[(1, 1, 1, 1), (150, 50), (4, 4), (221, 321)]

# Problem 5
- define a function decorator 'secure'
- secure adds two required arguments before any others, a 'user' and a 'password'
- if the user is not registered, raise an Exception
- if the user is registered, but the password is wrong, raise an Exception

In [98]:
def secure(func):
    def inner(user, pw, *args, **kwgs):
      
        if user in up.keys(): 
            if pw in up.values():
                l1 = list(args)
                l2 = list(kwgs.values())
                l = l1 + l2
                a = l[0]
                b = l[1]
                return(func(a, b))
            else:
                raise NameError('name %s is not defined' % user)
        elif user not in up.keys():
             raise Exception('User %s is not registered' % user)
    return (inner) # return the inner function (don't call it)

In [99]:
# the user/password 'database'
up = {}
up['jack'] = 'jackpw'
up['jill'] = 'jillpw'

    
@secure
def foo(a,b):
    return (a+b)

@secure
def bar(a, b=34):
    return(a+b)


In [100]:
# wrong number of args

foo(1,2)

Exception: User 1 is not registered

In [101]:
# good call

foo('jack', 'jackpw', 1 ,2)

3

In [102]:
# bad user

foo('frank', 'bad', 1 ,2)

Exception: User frank is not registered

In [103]:
# good user, bad passwd
foo('jack', 'nope')

NameError: name jack is not defined

In [104]:
# works with keywords

bar('jill', 'jillpw', 5, b=34)


39